# GLasso

In [1]:
# Settings
n = 20
p = 10
set.seed(0)
X_aux = matrix( rnorm(n*p), ncol=p ) # Auxillary data for generating truth of covariance
Sigma_true = cov(X_aux)
# Sigma_true
Theta_true = solve(Sigma_true)
Theta_true
X = MASS::mvrnorm(n=n, mu=rep(0,p), Sigma=Sigma_true)

1.27498518,-0.360013241,-0.13899649,-0.04053069,-0.108175498,-0.22897547,0.43052975,0.01709085,-0.39463630,0.02215637
-0.36001324,3.134190094,1.01737618,-0.77021533,-0.009104576,0.37902948,-0.44378911,0.56362750,0.11966498,-0.46572278
-0.13899649,1.017376182,1.38116548,-0.96414716,-0.030133155,0.05454111,-0.25264663,-0.01056241,0.05395003,-0.35386972
-0.04053069,-0.770215326,-0.96414716,3.52498851,0.656934400,-0.39441460,0.11595627,-0.58714774,0.03967201,-0.27566292
-0.10817550,-0.009104576,-0.03013316,0.65693440,1.878431591,0.14829337,-0.20895989,-0.30118765,0.03356446,-0.09224890
-0.22897547,0.379029484,0.05454111,-0.39441460,0.148293373,1.30772526,-0.04191247,0.11365076,-0.06444851,-0.14225305
0.43052975,-0.443789113,-0.25264663,0.11595627,-0.208959889,-0.04191247,1.32480422,-0.07923855,-0.22972662,-0.20156463
0.01709085,0.563627505,-0.01056241,-0.58714774,-0.301187650,0.11365076,-0.07923855,1.38270476,-0.05168962,0.31653789
-0.39463630,0.119664977,0.05395003,0.03967201,0.033564464,-0.06444851,-0.22972662,-0.05168962,0.93065700,-0.04880206
0.02215637,-0.465722778,-0.35386972,-0.27566292,-0.092248902,-0.14225305,-0.20156463,0.31653789,-0.04880206,1.77034700


In [2]:
rho = 0.1
t = 0.001 # a fixed threshold
S = cov(X)*(n-1)/n # empirical

In [3]:
ave_S = mean(S[lower.tri(S)])
threshold = t * ave_S
max_iter = 100

# Step 1: diagonal of W remains unchanged
W = S + rho*diag(p)
# print(W)
# B = matrix(0, (p-1), p) # store all betas

# Step 2:
iter = 0
while(iter < max_iter){
  W_old = W
  for(i in 1:p){ # iterate column in covariance
#     W11 = S[-i, -i] # use approximation as column-wise lasso for test
    W11 = W[-i, -i]
    
    s12 = S[-i,i]

    w12 = W[-i, i]
    beta = solve(W11) %*% w12
#     beta = B[, i]
    
    # coordinate decent
    max_iter_lasso = 20
    iter_lasso = 0
    while(iter_lasso < max_iter_lasso){
      beta_old = beta
      for(j in 1:(p-1)){ # iterate entries in beta
        uj = s12[j]
        for(k in 1:(p-1)){
          if(k==j) next
          uj = uj - W11[k,j] * beta[k]

        }
#         cat(j, uj, '  ')
        if(abs(uj) > rho){
          beta[j] = (abs(uj) - rho) / W11[j,j]
          if(uj < 0) beta[j] = -beta[j]
        } 
        else{
          beta[j] = 0
        }
      }
#       cat('\n')
      iter_lasso = iter_lasso + 1
      
      change_lasso = max( abs(beta - beta_old) )
      if(change_lasso<t) break
    }
#     cat('iter_lasso: ', iter_lasso, '\n')
    
    w12 = W11 %*% beta
#     cat(i, 'beta: ', beta, '\n', '  w12:', w12,'\n')
    W[-i, i] = W[i, -i] = w12
#     B[, i] = beta
#     print(W)
  }
  
  iter = iter + 1
  change = mean( abs(W-W_old) )
  if(change < threshold) break
  else{
    cat('iter: ', iter, change, '\n')
  }
}

iter:  1 0.06972078 
iter:  2 0.0007105458 
iter:  3 4.454189e-05 


In [4]:
solve(W)

1.566140e+00,-1.377575e-01,-7.057102e-07,-7.156436e-02,1.637889e-06,7.086835e-06,2.216490e-01,-4.407174e-02,-6.374834e-02,-8.015627e-02
-1.377575e-01,2.438288e+00,7.805919e-01,5.165979e-02,-9.711464e-07,4.852928e-06,1.283981e-01,1.227863e-01,-3.185270e-01,9.783597e-18
-7.057102e-07,7.805919e-01,1.007675e+00,-4.021857e-01,8.419648e-02,-5.867047e-07,-1.276214e-08,-3.098059e-01,-9.051479e-02,-2.336369e-17
-7.156436e-02,5.165979e-02,-4.021857e-01,2.754411e+00,-2.847759e-06,-3.335122e-06,-7.359494e-02,-9.551050e-02,1.435540e-07,-3.398458e-01
1.637889e-06,-9.711464e-07,8.419648e-02,-2.847759e-06,3.051068e+00,2.392397e-01,-1.998664e-01,1.112799e-06,1.697517e-07,-2.016825e-17
7.086835e-06,4.852928e-06,-5.867047e-07,-3.335122e-06,2.392397e-01,1.782954e+00,2.039395e-01,7.199483e-01,2.255606e-01,-2.201574e-01
2.216490e-01,1.283981e-01,-1.276214e-08,-7.359494e-02,-1.998664e-01,2.039395e-01,7.488347e-01,-2.595919e-01,-4.329613e-02,-3.340141e-01
-4.407174e-02,1.227863e-01,-3.098059e-01,-9.551050e-02,1.112799e-06,7.199483e-01,-2.595919e-01,1.712759e+00,-5.858777e-08,1.650320e-01
-6.374834e-02,-3.185270e-01,-9.051479e-02,1.435540e-07,1.697517e-07,2.255606e-01,-4.329613e-02,-5.858777e-08,8.058712e-01,2.517409e-02
-8.015627e-02,2.195599e-17,2.058374e-18,-3.398458e-01,-1.513931e-17,-2.201574e-01,-3.340141e-01,1.650320e-01,2.517409e-02,1.582096e+00


### Compare the outcome of original implementaion of glasso package (by Fortran)

In [5]:
glasso::glasso(S, rho=rho)$wi

1.56613964,-0.13775873,0.00000000,-0.07157370,0.00000000,0.0000000,0.22164804,-0.04407385,-0.06374970,-0.08015462
-0.13775247,2.43828099,0.78058897,0.05165581,0.00000000,0.0000000,0.12839691,0.12278872,-0.31852440,0.00000000
0.00000000,0.78059204,1.00767375,-0.40218915,0.08420825,0.0000000,0.00000000,-0.30980689,-0.09051454,0.00000000
-0.07156959,0.05165946,-0.40218376,2.75441401,0.00000000,0.0000000,-0.07359313,-0.09550780,0.00000000,-0.33984556
0.00000000,0.00000000,0.08420362,0.00000000,3.05106056,0.2392139,-0.19987311,0.00000000,0.00000000,0.00000000
0.00000000,0.00000000,0.00000000,0.00000000,0.23921260,1.7829465,0.20394242,0.71994355,0.22555987,-0.22015770
0.22164914,0.12839644,0.00000000,-0.07359884,-0.19987365,0.2039388,0.74883632,-0.25959247,-0.04329611,-0.33401443
-0.04407521,0.12277482,-0.30980495,-0.09550612,0.00000000,0.7199433,-0.25959402,1.71275680,0.00000000,0.16503252
-0.06375076,-0.31852412,-0.09051333,0.00000000,0.00000000,0.2255607,-0.04330004,0.00000000,0.80587060,0.02517474
-0.08015529,0.00000000,0.00000000,-0.33984512,0.00000000,-0.2201574,-0.33401508,0.16503224,0.02517434,1.58209656
